In [2]:
from get_stock_price import calc_ret_from_file
import os
import pandas as pd
import numpy as np
from build_model import *

In [46]:
df = concat_earning(100)
df['cap'] = df['cap'].map(lambda x: clean_cap(x))
df['est'] = df['est'].map(lambda x: clean_est(x))
df['report'] = df['report'].map(lambda x: clean_est(x))
df['surp'] = df['report'] - df['est']
df['pct_surp'] = df['report']/df['est'] - 1

before = df[df['timing'] == 'bmo']
before = add_ret_column(before,'day0ret', -1, 'adjusted_close', 0,'adjusted_close' )
before = add_ret_column(before,'overnight', -1, 'adjusted_close', 0,'open' )
before = add_ret_column(before, 'day0close_open', 0,'open',0,'adjusted_close')
before = add_ret_column(before,  'ret 3',0, 'open', 2,'adjusted_close')
before = add_price_column(before,  'day0open',0, 'open')

after = df[df['timing'] == 'amc']
after = add_ret_column(after, 'day0ret', 0,'adjusted_close', 1,'adjusted_close')
after = add_ret_column(after, 'overnight', 0,'adjusted_close', 1,'open')
after = add_ret_column(after, 'day0close_open',1, 'open', 1,'adjusted_close')
after = df = add_ret_column(after, 'ret 3',1, 'open', 3,'adjusted_close')
after = add_price_column(after,  'day0open',1, 'open')


cob = pd.concat([before, after])
cob = cob.dropna()

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

In [47]:
import seaborn as sns
import matplotlib.pyplot as plt

In [48]:
cob['sur_cap'] = cob['surp']/cob['cap']
cob = cob[cob['day0open'] >= 5]

In [49]:
cob.describe()

cap         est      report        surp    pct_surp  \
count  7.800000e+02  780.000000  780.000000  780.000000  780.000000   
mean   3.584211e+04    0.844692    0.894859    0.050167         NaN   
std    9.935784e+04    1.480009    1.696044    0.362250         NaN   
min    1.600000e+01  -13.000000  -19.970000   -6.970000        -inf   
25%    1.244750e+03    0.170000    0.190000    0.000000   -0.022617   
50%    5.621000e+03    0.540000    0.580000    0.030000    0.037506   
75%    2.630775e+04    1.202500    1.242500    0.100000    0.138846   
max    1.046894e+06   19.390000   22.490000    3.100000         inf   

          day0ret   overnight  day0close_open       ret 3     day0open  \
count  780.000000  780.000000      780.000000  780.000000   780.000000   
mean    -0.006016   -0.000050       -0.005279   -0.008629    77.695126   
std      0.089842    0.085637        0.054141    0.065944   144.488464   
min     -0.571673   -0.432324       -0.499759   -0.497641     5.000000   
25%     -0.048495   -0.033692       -0.031346   -0.043309    18.912500   
50%     -0.001103    0.003516       -0.004599   -0.008463    43.270000   
75%      0.037024    0.037786        0.021913    0.026524    94.417500   
max      0.543614    1.099921        0.217330    0.259301  1929.000000   

          sur_cap  
count  780.000000  
mean    -0.000549  
std      0.015607  
min     -0.435625  
25%      0.000000  
50%      0.000002  
75%      0.000020  
max      0.004114

In [50]:
cob.corr()

cap       est    report      surp  pct_surp   day0ret  \
cap             1.000000  0.310067  0.306653  0.168928  0.022857  0.037710   
est             0.310067  1.000000  0.983158  0.517512  0.040668  0.045126   
report          0.306653  0.983158  1.000000  0.665179  0.051436  0.061586   
surp            0.168928  0.517512  0.665179  1.000000  0.074663  0.103976   
pct_surp        0.022857  0.040668  0.051436  0.074663  1.000000  0.031254   
day0ret         0.037710  0.045126  0.061586  0.103976  0.031254  1.000000   
overnight       0.041603  0.060308  0.084317  0.148374  0.019064  0.746444   
day0close_open -0.005126 -0.034890 -0.048336 -0.083760  0.023657  0.518902   
ret 3           0.022291  0.012126  0.002212 -0.039188 -0.003014  0.409513   
day0open        0.437815  0.723063  0.712558  0.382026  0.041044  0.026557   
sur_cap         0.012740  0.334842  0.442651  0.704450 -0.010129  0.037682   

                overnight  day0close_open     ret 3  day0open   sur_cap  
cap              0.041603       -0.005126  0.022291  0.437815  0.012740  
est              0.060308       -0.034890  0.012126  0.723063  0.334842  
report           0.084317       -0.048336  0.002212  0.712558  0.442651  
surp             0.148374       -0.083760 -0.039188  0.382026  0.704450  
pct_surp         0.019064        0.023657 -0.003014  0.041044 -0.010129  
day0ret          0.746444        0.518902  0.409513  0.026557  0.037682  
overnight        1.000000       -0.148413 -0.124963  0.026190  0.101045  
day0close_open  -0.148413        1.000000  0.800899  0.002354 -0.125289  
ret 3           -0.124963        0.800899  1.000000  0.028603 -0.060577  
day0open         0.026190        0.002354  0.028603  1.000000  0.017750  
sur_cap          0.101045       -0.125289 -0.060577  0.017750  1.000000

In [51]:
import statsmodels.api as sm

In [52]:
x = sm.add_constant(cob[['overnight','sur_cap']])
y = list(cob['day0close_open'])
lm = sm.OLS(y,x).fit()

D:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [53]:
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     13.81
Date:                Mon, 29 Jul 2019   Prob (F-statistic):           1.28e-06
Time:                        23:47:35   Log-Likelihood:                 1182.0
No. Observations:                 780   AIC:                            -2358.
Df Residuals:                     777   BIC:                            -2344.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0055      0.002     -2.879      0.0

In [54]:
cob.sort_values(['day0close_open'],ascending = False)

symbol             company      cap timing    est  report  surp  \
2019-06-26    RAD  Rite Aid Corpor...    365.0    amc   0.02   -0.14 -0.16   
2019-05-08   ROKU          Roku, Inc.  12119.0    amc  -0.24   -0.09  0.15   
2019-06-26    PIR  Pier 1 Imports,...     16.0    amc -13.00  -19.97 -6.97   
2019-06-18    LZB  La-z-boy Incorp...   1517.0    amc   0.65    0.64 -0.01   
2019-05-22    MOD  Modine Manufact...    727.0    amc   0.38    0.40  0.02   
2019-03-04   LJPC  La Jolla Pharma...    236.0    amc  -1.76   -1.73  0.03   
2019-05-02   ABMD       Abiomed, Inc.  11795.0    bmo   1.07    1.07  0.00   
2019-03-04     YY             Yy Inc.   4105.0    amc   1.78    1.87  0.09   
2019-06-05   SMAR  Smartsheet Inc....   5673.0    amc  -0.18   -0.12  0.06   
2019-05-21     DY  Dycom Industrie...   1672.0    bmo   0.43    0.53  0.10   
2019-03-25    YRD       Yirendai Ltd.    701.0    bmo   0.51    1.32  0.81   
2019-05-21    SKY  Skyline Corpora...   1544.0    amc   0.21    0.26  0.05   
2019-02-28   EDIT  Editas Medicine...   1174.0    amc  -0.48   -0.52 -0.04   
2019-05-16   VNET  21vianet Group,...    799.0    amc   0.03    0.01 -0.02   
2019-04-26   HELE  Helen Of Troy L...   3715.0    bmo   1.58    1.82  0.24   
2019-05-21     DQ  Daqo New Energy...    478.0    bmo   0.33    0.81  0.48   
2019-05-02      X  United States S...   2598.0    amc   0.17    0.47  0.30   
2019-03-06   CDNA        Caredx, Inc.   1336.0    amc   0.01    0.01  0.00   
2019-04-29   HLIT       Harmonic Inc.    522.0    amc  -0.06   -0.05  0.01   
2019-07-10   PSMT  Pricesmart, Inc...   1884.0    amc   0.42    0.55  0.13   
2019-06-03    BOX           Box, Inc.   2454.0    amc  -0.05   -0.03  0.02   
2019-02-28   SGMO  Sangamo Therape...   1338.0    amc  -0.19   -0.18  0.01   
2019-06-27    GMS            Gms Inc.    905.0    bmo   0.71    0.68 -0.03   
2019-05-17    CAE             Cae Inc   7165.0    bmo   0.33    0.36  0.03   
2019-05-09    HSC  Harsco Corporat...   2010.0    bmo   0.24    0.29  0.05   
2019-05-15   RUBY  Rubius Therapeu...   1205.0    bmo  -0.38   -0.42 -0.04   
2019-06-20    MEI  Methode Electro...   1067.0    bmo   0.71    0.60 -0.11   
2019-03-06   YEXT           Yext Inc.   2288.0    amc  -0.09   -0.03  0.06   
2019-04-09   PSMT  Pricesmart, Inc...   1884.0    amc   0.56    0.88  0.32   
2019-05-06    BHC  Bausch Health C...   8157.0    bmo   0.89    1.03  0.14   
...           ...                 ...      ...    ...    ...     ...   ...   
2019-04-30   TWLO         Twilio Inc.  18387.0    amc   0.01    0.05  0.04   
2019-02-26    MED        Medifast Inc   1353.0    amc   1.19    1.30  0.11   
2019-03-13   CLDR      Cloudera, Inc.   1552.0    amc  -0.11   -0.15 -0.04   
2019-05-02   ONDK  On Deck Capital...    301.0    bmo   0.11    0.10 -0.01   
2019-05-07   CROX         Crocs, Inc.   1662.0    bmo   0.25    0.36  0.11   
2019-06-20    KFY  Korn/ferry Inte...   2215.0    amc   0.89    0.88 -0.01   
2019-03-21   CSIQ   Canadian Solar...   1245.0    bmo   0.87    1.61  0.74   
2019-03-04   PKOH  Park-ohio Holdi...    385.0    amc   0.88    1.20  0.32   
2019-05-09   CRON  Cronos Group In...   5032.0    bmo  -0.02    0.36  0.38   
2019-04-29    MDR  Mcdermott Inter...   1662.0    bmo   0.09    0.02 -0.07   
2019-03-05   CIEN  Ciena Corporati...   6675.0    bmo   0.30    0.33  0.03   
2019-05-08   MEET        Meetme, Inc.    267.0    bmo   0.09    0.09  0.00   
2019-03-01    SNH   Senior Housing...   1926.0    bmo   0.37    0.27 -0.10   
2019-02-27    GNL  Global Net Leas...   1577.0    bmo   0.54    0.50 -0.04   
2019-05-09   INTT  Intest Corporat...     48.0    amc   0.16    0.14 -0.02   
2019-05-06    LCI      Lannett Co Inc    235.0    amc   0.62    0.68  0.06   
2019-05-21   OOMA          Ooma, Inc.    247.0    amc  -0.06   -0.04  0.02   
2019-03-05    GMS            Gms Inc.    905.0    bmo   0.54    0.41 -0.13   
2019-05-28     JT  Jianpu Technolo...    612.0    bmo  -0.02    0.03  0.05   
2019-05-22   PLUS          Eplus 